image.png

In [1]:
import tensorflow as tf
import numpy as np # 추가
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report # 추가


2025-11-25 02:36:43.123156: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 02:36:45.685442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 02:37:06.291367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:

# 사용자 설정
train_dir = '/workspace/merge_data/data/img/train'
val_dir = '/workspace/merge_data/data/img/val'
target_size = (224, 224)
batch_size = 32
seed = 42

# Data generators (EfficientNet 전처리 사용)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 다중 클래스 분류: class_mode='categorical'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',  # 변경: categorical (one-hot)
    seed=seed
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',  # 변경: categorical
    shuffle=False,             # 평가 시 순서 유지
    seed=seed
)

print("Class indices:", train_generator.class_indices)
class_names = list(train_generator.class_indices.keys())
print("Class names (in order):", class_names)



Found 5841 images belonging to 4 classes.
Found 1151 images belonging to 4 classes.
Class indices: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3}
Class names (in order): ['anger', 'happy', 'panic', 'sadness']


In [3]:
# Base model 로드 (EfficientNetB0)
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

# top 추가: softmax 출력 (클래스 수 = 4)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(class_names), activation='softmax')(x)  # 다중 클래스

model = Model(inputs=base_model.input, outputs=predictions)

# base freeze
for layer in base_model.layers:
    layer.trainable = False

# 컴파일: categorical_crossentropy 사용
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

model.summary()

# 안전한 step 계산
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

# 학습
print("\n--- 모델 학습 시작 ---")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
)
print("--- 모델 학습 완료 ---\n")

# --- 검증 데이터셋 성능 평가 ---
print("--- 검증 데이터셋 성능 평가 시작 ---")

# 1. 실제 레이블 (정수 인덱스) 추출
# flow_from_directory의 경우 .classes 속성이 존재 (정수 레이블)
if hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
else:
    # 안전장치: one-hot -> argmax
    validation_labels = np.argmax(validation_generator.labels, axis=1)

# 2. 예측 (확률)
validation_generator.reset()
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)

# 3. 확률 -> 정수 클래스 (argmax)
pred_classes = np.argmax(pred_probabilities, axis=1)

# 4. classification_report 출력 (정수 레이블 사용)
# class_names는 train_generator.class_indices의 키 순서와 일치
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)
print("--- 검증 데이터셋 성능 평가 완료 ---")



I0000 00:00:1764038262.902396  649942 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42799 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,214,055 (16.08 MB)

 Trainable params: 164,484 (642.52 KB)

...

  4/183 ━━━━━━━━━━━━━━━━━━━━ 3:45 1s/step - accuracy: 0.3034 - loss: 1.5045 - top3_acc: 0.7051  

2025-11-25 02:38:53.689141: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:38:53.815374: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:38:54.228907: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:38:54.356956: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:38:54.913529: E external/local_xla/xla/stream_

183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3020 - loss: 1.4049 - top3_acc: 0.7871

2025-11-25 02:46:55.240816: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:46:55.367755: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:46:55.762643: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:46:55.889471: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-25 02:46:56.385436: E external/local_xla/xla/stream_

183/183 ━━━━━━━━━━━━━━━━━━━━ 537s 3s/step - accuracy: 0.3292 - loss: 1.3600 - top3_acc: 0.8139 - val_accuracy: 0.3440 - val_loss: 1.3399 - val_top3_acc: 0.8540
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 472s 2s/step - accuracy: 0.4143 - loss: 1.2637 - top3_acc: 0.8713 - val_accuracy: 0.3545 - val_loss: 1.3750 - val_top3_acc: 0.8480
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 398s 2s/step - accuracy: 0.4568 - loss: 1.2115 - top3_acc: 0.8868 - val_accuracy: 0.3788 - val_loss: 1.3268 - val_top3_acc: 0.8653
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 411s 2s/step - accuracy: 0.4751 - loss: 1.1776 - top3_acc: 0.9004 - val_accuracy: 0.3901 - val_loss: 1.3237 - val_top3_acc: 0.8705
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 398s 2s/step - accuracy: 0.4970 - loss: 1.1367 - top3_acc: 0.9087 - val_accuracy: 0.3927 - val_loss: 1.3263 - val_top3_acc: 0.8801
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 387s 2s/step - accuracy: 0.5210 - loss: 1.1037 - top3_acc: 0.9218 - val_accuracy: 0.4083 - val_loss: 1.3698 - val_to

In [5]:
import numpy as np
from sklearn.metrics import classification_report

# 안전한 클래스 이름 복원 (index 순서대로)
class_indices = train_generator.class_indices  # {'class_name': index, ...}
inv_map = {v: k for k, v in class_indices.items()}
class_names = [inv_map[i] for i in range(len(inv_map))]

# 실제 레이블(정수 인덱스) 안전하게 가져오기
if hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
elif hasattr(validation_generator, 'labels'):
    # flow_from_dataframe 등에서 one-hot 또는 raw일 경우 처리
    labels_attr = validation_generator.labels
    if isinstance(labels_attr, np.ndarray) and labels_attr.ndim == 2:
        validation_labels = np.argmax(labels_attr, axis=1)
    else:
        validation_labels = np.array(labels_attr)
else:
    # 안전장치: validation_generator를 flow_from_directory로 만들었다면 val_dir 스캔으로 복원 가능
    raise RuntimeError("Cannot find validation labels in the generator. Ensure generator provides .classes or .labels.")

# 예측 수행 (제너레이터 초기화 후)
validation_generator.reset()
validation_steps = len(validation_generator)
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)

# 확률 -> 정수 클래스
pred_classes = np.argmax(pred_probabilities, axis=1)

# classification_report 출력
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)



36/36 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step

--- Classification Report (Validation Set) ---

              precision    recall  f1-score   support

       anger     0.3099    0.2577    0.2814       291
       happy     0.4196    0.4448    0.4318       299
       panic     0.4206    0.5393    0.4726       280
     sadness     0.3562    0.2954    0.3230       281

    accuracy                         0.3840      1151
   macro avg     0.3766    0.3843    0.3772      1151
weighted avg     0.3766    0.3840    0.3771      1151

